In [6]:
import os
import cv2
from PIL import Image
import numpy as np
import pickle as pkl
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from processer import equalize, zmIceColor, extractGreen
from dataset import PlantSeedDataset
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split

In [7]:
datasets_path = "../dataset/classifer/train"
    
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        np.asarray,
        transforms.Lambda(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
        ])
dataset = PlantSeedDataset(datasets_path, transform=transform)

In [8]:
hog_pca = np.load('hog_pca.npy')

In [9]:
hog_pca.shape

(4440, 100)

In [18]:
with open('hog_features.pkl', 'rb') as f:
    hog_features = pkl.load(f)

In [20]:
hog_features[0].shape

(129600,)

In [21]:
X_train,X_test, y_train, y_test =train_test_split(hog_features, dataset.labels, test_size=0.2, random_state=0)

In [22]:
model_xgb = XGBClassifier(learning_rate=0.05, 
                          objective='binary:logistic',
                          booster="gbtree",
                          num_class=12, 
                          n_estimators=1000,
                          tree_method="hist",
                          device="cuda",
                          use_label_encoder=False)
# model_xgb.fit(X_train, y_train)
model_xgb.fit(X_train, 
              y_train, 
              early_stopping_rounds=10, 
              eval_set=[(X_test, y_test)], 
              eval_metric='mlogloss', 
              verbose=50)

/datas/wangpeidong/.conda/envs/pytorch/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/datas/wangpeidong/.conda/envs/pytorch/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.46298
[50]	validation_0-mlogloss:2.01916
[100]	validation_0-mlogloss:1.92174
[150]	validation_0-mlogloss:1.89805
[165]	validation_0-mlogloss:1.89610


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, num_class=12,
              num_parallel_tree=None, ...)

In [24]:
model_xgb.save_model("xgb_hog.json")

In [25]:
import sklearn.metrics
y_pred = model_xgb.predict(X_test)
sklearn.metrics.f1_score(y_test, y_pred, average='micro')

0.36036036036036034

In [27]:
y_test

[11,
 0,
 1,
 8,
 5,
 3,
 7,
 5,
 11,
 11,
 6,
 8,
 2,
 2,
 11,
 1,
 5,
 6,
 6,
 2,
 11,
 6,
 6,
 8,
 8,
 8,
 6,
 4,
 4,
 6,
 3,
 3,
 11,
 1,
 1,
 4,
 6,
 6,
 2,
 8,
 11,
 3,
 4,
 3,
 3,
 1,
 5,
 11,
 5,
 3,
 11,
 6,
 10,
 6,
 2,
 2,
 3,
 7,
 7,
 3,
 6,
 0,
 6,
 11,
 8,
 9,
 10,
 11,
 10,
 1,
 11,
 11,
 1,
 9,
 3,
 0,
 6,
 1,
 7,
 4,
 11,
 8,
 10,
 2,
 1,
 2,
 3,
 3,
 2,
 10,
 7,
 10,
 4,
 2,
 5,
 5,
 2,
 7,
 0,
 6,
 5,
 8,
 3,
 5,
 6,
 10,
 6,
 5,
 0,
 7,
 4,
 6,
 1,
 6,
 0,
 8,
 7,
 2,
 3,
 8,
 9,
 10,
 1,
 5,
 10,
 5,
 1,
 11,
 2,
 9,
 11,
 1,
 7,
 6,
 9,
 0,
 3,
 10,
 5,
 5,
 5,
 8,
 10,
 6,
 9,
 3,
 8,
 6,
 4,
 4,
 6,
 6,
 10,
 6,
 5,
 6,
 6,
 6,
 10,
 6,
 6,
 0,
 6,
 6,
 5,
 11,
 8,
 1,
 10,
 5,
 8,
 1,
 5,
 11,
 5,
 4,
 0,
 11,
 10,
 10,
 5,
 10,
 0,
 8,
 8,
 1,
 11,
 3,
 8,
 11,
 1,
 10,
 4,
 3,
 3,
 1,
 2,
 8,
 1,
 10,
 10,
 10,
 1,
 7,
 6,
 5,
 0,
 0,
 6,
 2,
 3,
 1,
 2,
 9,
 8,
 10,
 6,
 11,
 3,
 6,
 9,
 11,
 1,
 1,
 6,
 5,
 1,
 2,
 11,
 10,
 10,
 3,
 5,
 3,
 8,
 10,
 6,
 1,

In [26]:
y_pred

array([ 6,  0,  3,  6,  1,  3,  8, 10, 10, 11,  8,  8,  3,  2,  0,  5,  5,
        6,  6,  2,  8,  6,  6,  8,  6,  8,  3,  3,  6, 11, 10,  3, 11,  5,
        8,  6,  6,  6,  3,  6,  4,  3,  1,  3,  3,  3,  5,  8,  5,  3, 10,
        2, 10,  6,  2,  2,  3,  8, 10,  3,  6,  8,  6,  7,  8,  2, 10, 11,
        3,  5, 11,  0,  3,  2,  6, 10,  6,  3,  5,  9,  6,  6,  3,  2,  5,
        2,  3,  3,  2,  5,  7, 10,  3,  2,  8,  8,  0,  8,  6,  6,  5,  3,
        3, 10,  6,  6,  8,  5,  2,  8,  3,  6, 10,  6,  3,  5,  6,  0,  5,
       10,  3,  8, 10,  8,  6,  5,  8, 10,  2,  6, 11,  3,  8,  6,  6,  8,
        3,  3,  5,  5,  3,  8, 10,  6,  3,  3,  3,  6,  3,  8,  6,  6,  8,
        3,  5,  6,  6, 10,  5,  6,  6,  8,  6,  6,  1,  6,  8,  7, 10,  3,
        8,  6,  1,  7,  5,  5,  6,  6,  3,  8,  1,  5, 11,  8,  6,  3,  3,
        3,  3, 10, 10,  5,  3,  3,  5,  3, 10,  8,  5,  3, 10,  5,  1,  4,
        6, 10,  6, 10,  6,  3,  3, 10,  3,  5,  6,  3,  8, 10,  6, 11,  2,
        3,  8,  5,  6,  5